In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import linear_model, metrics
from sklearn.linear_model import Lasso

In [3]:
data = pd.read_csv("./chevron.csv")
# Clean data:
# get rid of zeros
# make sure that empty cells is filled in with 0
# drop non states (US, etc.)
# lots of 0's
# get rid of random features that are duplicates
# drop conversion factor columns

## Filter stuff out
# states
## dropping DC, US, Gulf of Mexico, 
## factors

## Convert stuff to billion btu
# fuel ethanol

# add additional data
## add CO2 emissions data
## some other stuff?
## aggregate data?

# normalize the features
# adjust the weights
# LASSO, some other method

# other thoughts:
## LP normalization: is this what we want?
## if everything is in the same unit... then do we need to normalize?
## 

In [4]:
print(data.head())
print(data.shape)

   Unnamed: 0    MSN StateCode  Year   Amount   State  CO2 Emissions (Mmt)  \
0           0  BDFDB        AK  2015     21.0  Alaska            35.027804   
1           1  BDPRP        AK  2015      4.0  Alaska            35.027804   
2           2  BFFDB        AK  2015     21.0  Alaska            35.027804   
3           3  BFPRP        AK  2015      4.0  Alaska            35.027804   
4           4  CLPRB        AK  2015  17747.0  Alaska            35.027804   

   TotalNumberofInvestments  TotalAmountofAssistance  
0                      16.0                3345612.0  
1                      16.0                3345612.0  
2                      16.0                3345612.0  
3                      16.0                3345612.0  
4                      16.0                3345612.0  
(7595, 9)


In [5]:
# Clean the data
# Drop unnecessary rows

In [6]:
# Drop DC, US, Gulf of Mexico, Pacific
clean_df = data.query("StateCode != 'DC'").query("StateCode != 'US'").query("StateCode != 'X3'").query("StateCode != 'X5'")
print(clean_df.head())
print(clean_df.shape)
print(clean_df.StateCode.unique())
print(clean_df.StateCode.unique().size)

   Unnamed: 0    MSN StateCode  Year   Amount   State  CO2 Emissions (Mmt)  \
0           0  BDFDB        AK  2015     21.0  Alaska            35.027804   
1           1  BDPRP        AK  2015      4.0  Alaska            35.027804   
2           2  BFFDB        AK  2015     21.0  Alaska            35.027804   
3           3  BFPRP        AK  2015      4.0  Alaska            35.027804   
4           4  CLPRB        AK  2015  17747.0  Alaska            35.027804   

   TotalNumberofInvestments  TotalAmountofAssistance  
0                      16.0                3345612.0  
1                      16.0                3345612.0  
2                      16.0                3345612.0  
3                      16.0                3345612.0  
4                      16.0                3345612.0  
(7250, 9)
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA

In [7]:
# Convert everything to billion btu
## Convert biodiesel to billion btu
clean_df = clean_df
# Remove the BDFDB, BFFDB, BFPRP, EMFDB
clean_df = clean_df.query("MSN != 'BDFDB'").query("MSN != 'BFFDB'").query("MSN != 'BFPRP'").query("MSN != 'EMFDB'")
print(clean_df.shape)
# Convert biodiesel to energy
biodiesel_converted = clean_df.query("MSN == 'BDPRP'")
# print(biodiesel_converted['Amount'])
biodiesel_converted.loc[:, "Amount"] = biodiesel_converted.loc[:, "Amount"] * 5.46
print(biodiesel_converted)
non_bdprp = clean_df.query("MSN != 'BDPRP'")
clean_df = pd.concat([non_bdprp, biodiesel_converted])
print(clean_df.shape)


(6250, 9)
      Unnamed: 0    MSN StateCode  Year   Amount          State  \
1              1  BDPRP        AK  2015    21.84         Alaska   
30            30  BDPRP        AL  2015  1943.76        Alabama   
59            59  BDPRP        AR  2015  4449.90       Arkansas   
88            88  BDPRP        AZ  2015    10.92        Arizona   
117          117  BDPRP        CA  2015  4078.62     California   
...          ...    ...       ...   ...      ...            ...   
7440        7440  BDPRP        VT  2019     0.00        Vermont   
7469        7469  BDPRP        WA  2019  9871.68     Washington   
7498        7498  BDPRP        WI  2019  3510.78      Wisconsin   
7527        7527  BDPRP        WV  2019     0.00  West Virginia   
7556        7556  BDPRP        WY  2019     0.00        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
1               35.027804                      16.0                3345612.0  
30             119.200889  

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [8]:
# Convert ethanol to energy
ethanol_converted = clean_df.query("MSN == 'ENPRP'")
print(ethanol_converted['Amount'])
ethanol_converted.loc[:, "Amount"] = ethanol_converted.loc[:, "Amount"] * 3.192
print(ethanol_converted['Amount'])
non_enprp = clean_df.query("MSN != 'ENPRP'")
clean_df = pd.concat([non_enprp, ethanol_converted])
print(clean_df.shape)

9           0.0
38          0.0
67          0.0
96       1154.0
125      4650.0
         ...   
7448        0.0
7477        0.0
7506    15000.0
7535        0.0
7564        0.0
Name: Amount, Length: 250, dtype: float64
9           0.000
38          0.000
67          0.000
96       3683.568
125     14842.800
          ...    
7448        0.000
7477        0.000
7506    47880.000
7535        0.000
7564        0.000
Name: Amount, Length: 250, dtype: float64
(6250, 9)


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [9]:
clean_df = clean_df.query("MSN != 'CLPRK'").query("MSN != 'CLPRP'").query("MSN != 'COPRK'") \
                    .query("MSN != 'PAPRP'").query("MSN != 'NGMPK'").query("MSN != 'NGMPP'")
print(clean_df)


      Unnamed: 0    MSN StateCode  Year    Amount          State  \
4              4  CLPRB        AK  2015   17747.0         Alaska   
10            10  GETCB        AK  2015     186.0         Alaska   
11            11  HYTCB        AK  2015   14621.0         Alaska   
12            12  NCPRB        AK  2015   16298.0         Alaska   
13            13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
7448        7448  ENPRP        VT  2019       0.0        Vermont   
7477        7477  ENPRP        WA  2019       0.0     Washington   
7506        7506  ENPRP        WI  2019   47880.0      Wisconsin   
7535        7535  ENPRP        WV  2019       0.0  West Virginia   
7564        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
4               35.027804                      16.0                3345612.0  
10              35.027804

In [11]:
# Sort and reindex dataframe
print(clean_df.sort_values('Unnamed: 0'))
clean_df.reset_index(drop=True, inplace=True)

      Unnamed: 0    MSN StateCode  Year    Amount    State  \
1              1  BDPRP        AK  2015     21.84   Alaska   
4              4  CLPRB        AK  2015  17747.00   Alaska   
9              9  ENPRP        AK  2015      0.00   Alaska   
10            10  GETCB        AK  2015    186.00   Alaska   
11            11  HYTCB        AK  2015  14621.00   Alaska   
...          ...    ...       ...   ...       ...      ...   
7579        7579  WDPRB        WY  2019   4969.00  Wyoming   
7580        7580  WDTCB        WY  2019   4969.00  Wyoming   
7581        7581  WSTCB        WY  2019      0.00  Wyoming   
7582        7582  WWPRB        WY  2019   4969.00  Wyoming   
7583        7583  WYTCB        WY  2019  37070.00  Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
1               35.027804                      16.0                3345612.0  
4               35.027804                      16.0                3345612.0  
9               35

In [12]:
print(clean_df)

      Unnamed: 0    MSN StateCode  Year    Amount          State  \
0              4  CLPRB        AK  2015   17747.0         Alaska   
1             10  GETCB        AK  2015     186.0         Alaska   
2             11  HYTCB        AK  2015   14621.0         Alaska   
3             12  NCPRB        AK  2015   16298.0         Alaska   
4             13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
4745        7448  ENPRP        VT  2019       0.0        Vermont   
4746        7477  ENPRP        WA  2019       0.0     Washington   
4747        7506  ENPRP        WI  2019   47880.0      Wisconsin   
4748        7535  ENPRP        WV  2019       0.0  West Virginia   
4749        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804

In [69]:
# See how many zeroes there are in the dataframe
zeros_df = clean_df.query("Amount == 0")
print(zeros_df)

      Unnamed: 0    MSN StateCode  Year  Amount          State  \
16            16  NUETB        AK  2015     0.0         Alaska   
23            23  WDEXB        AK  2015     0.0         Alaska   
52            52  WDEXB        AL  2015     0.0        Alabama   
57            57  WYTCB        AL  2015     0.0        Alabama   
81            81  WDEXB        AR  2015     0.0       Arkansas   
...          ...    ...       ...   ...     ...            ...   
7390        7390  ENPRP        UT  2019     0.0           Utah   
7448        7448  ENPRP        VT  2019     0.0        Vermont   
7477        7477  ENPRP        WA  2019     0.0     Washington   
7535        7535  ENPRP        WV  2019     0.0  West Virginia   
7564        7564  ENPRP        WY  2019     0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
16              35.027804                      16.0                3345612.0  
23              35.027804                      16

In [29]:
# Test Query Code
test_query = clean_df.query("StateCode == 'AK' & Year == 2015")
print(test_query)
print(test_query.iloc[0, 8])
print(type(test_query.iloc[0, 8]))

# See how many features we have remaining
print(clean_df.MSN.unique())
print(clean_df.MSN.unique().size)

      Unnamed: 0    MSN StateCode  Year      Amount   State  \
0              4  CLPRB        AK  2015    17747.00  Alaska   
1             10  GETCB        AK  2015      186.00  Alaska   
2             11  HYTCB        AK  2015    14621.00  Alaska   
3             12  NCPRB        AK  2015    16298.00  Alaska   
4             13  NGMPB        AK  2015   380887.00  Alaska   
5             16  NUETB        AK  2015        0.00  Alaska   
6             17  PAPRB        AK  2015  1007501.00  Alaska   
7             19  REPRB        AK  2015    23807.00  Alaska   
8             20  SOTCB        AK  2015        3.00  Alaska   
9             21  TEPRB        AK  2015  1429942.00  Alaska   
10            22  TETCB        AK  2015   614242.00  Alaska   
11            23  WDEXB        AK  2015        0.00  Alaska   
12            24  WDPRB        AK  2015     6865.00  Alaska   
13            25  WDTCB        AK  2015     6865.00  Alaska   
14            26  WSTCB        AK  2015      622.00  Al

In [33]:
# alaska_df = clean_df.loc[clean_df["StateCode"] == "AK"]
print(clean_df)
print(clean_df.StateCode.unique())
print(clean_df.StateCode.unique().size)

      Unnamed: 0    MSN StateCode  Year    Amount          State  \
0              4  CLPRB        AK  2015   17747.0         Alaska   
1             10  GETCB        AK  2015     186.0         Alaska   
2             11  HYTCB        AK  2015   14621.0         Alaska   
3             12  NCPRB        AK  2015   16298.0         Alaska   
4             13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
4745        7448  ENPRP        VT  2019       0.0        Vermont   
4746        7477  ENPRP        WA  2019       0.0     Washington   
4747        7506  ENPRP        WI  2019   47880.0      Wisconsin   
4748        7535  ENPRP        WV  2019       0.0  West Virginia   
4749        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804

In [34]:
### Test on all states
# Generate all years and all states
all_years = [2015, 2016, 2017, 2018, 2019]
all_states = clean_df.StateCode.unique()

In [39]:
print(clean_df)
print(all_states)

      Unnamed: 0    MSN StateCode  Year    Amount          State  \
0              4  CLPRB        AK  2015   17747.0         Alaska   
1             10  GETCB        AK  2015     186.0         Alaska   
2             11  HYTCB        AK  2015   14621.0         Alaska   
3             12  NCPRB        AK  2015   16298.0         Alaska   
4             13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
4745        7448  ENPRP        VT  2019       0.0        Vermont   
4746        7477  ENPRP        WA  2019       0.0     Washington   
4747        7506  ENPRP        WI  2019   47880.0      Wisconsin   
4748        7535  ENPRP        WV  2019       0.0  West Virginia   
4749        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804

In [75]:
# Create Response Variable

num_datapoints = len(all_years) * len(all_states) 
num_cols = 1
y = pd.DataFrame(index=range(num_datapoints), columns=["TotalAmountofAssistance"])
investment_vals = []
for year in all_years:
    for state in all_states:
        state_query = "StateCode == " + "'" + str(state) + "'"
        year_query = "Year == " + str(year)
        investment_year_state = clean_df.query(year_query + " & " + state_query)
        # print("INVESTMENT YEAR QUERY DF")
        # print(investment_year.head())
        # print(investment_year.shape)
        investment_year_val = investment_year_state.iloc[0, 8]
        investment_vals.append(investment_year_val)
for yIdx in range(num_datapoints):
    y.at[yIdx, "TotalAmountofAssistance"] = investment_vals[yIdx]
# y.at[0]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][0]
# print(y)
# y.at[1]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][1519]
# y.at[2]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][3038]
# y.at[3]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][4557]
# y.at[4]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][6076]
# for investment_idx in range(5):
#     print(investment_idx)
#     print(data["TotalAmountofAssistance"][3038])
#     row_idx = 1519 * investment_idx
#     y.at[investment_idx]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][row_idx]
print(y)

y.to_csv('assistance.csv')

    TotalAmountofAssistance
0                 3345612.0
1                 1864412.0
2                13453887.0
3                 1724853.0
4                30077512.0
..                      ...
245               1121855.0
246              18389113.0
247               2915140.0
248                790383.0
249                570570.0

[250 rows x 1 columns]


In [64]:
# num_datapoints = len(all_years) * len(all_states) 
# num_features = 19

# all_msns = clean_df.MSN.unique()

# X = pd.DataFrame(index=range(num_datapoints), columns=all_msns)

# for yearIdx in range(len(all_years)):
#     for stateIdx in range(len(all_states)):
#         dataPtIdx = stateIdx * (yearIdx + 1)
#         # print(dataPtIdx)
#         curYear = all_years[yearIdx]
#         curState = all_states[stateIdx]
#         for msnIdx in range(len(all_msns)):
#             state_query = "StateCode == " + "'" + str(curState) + "'"
#             year_query = "Year == " + str(curYear)
#             msn_query = "MSN == " + "'" + str(all_msns[msnIdx]) + "'"
#             state_year_msn = clean_df.query(year_query + " & " + state_query + " & " + msn_query)
#             # print(state_year_msn)
#             amount_val = state_year_msn.iloc[0, 4]
#             X.at[dataPtIdx, all_msns[msnIdx]] = amount_val

# print(X)

         CLPRB  GETCB     HYTCB     NCPRB      NGMPB     NUETB     PAPRB  \
0      14867.0  186.0   14456.0   15951.0   364363.0       0.0  968358.0   
1     331420.0  141.0   91902.0   92144.0   182481.0  438728.0   55421.0   
2     247632.0  141.0   64484.0   65056.0   176290.0  417334.0   46417.0   
3     326748.0  141.0   85103.0   87057.0   159686.0  446097.0   39100.0   
4     370533.0  141.0  101447.0  105013.0   149607.0  412584.0   33591.0   
..         ...    ...       ...       ...        ...       ...       ...   
245  4828513.0  663.0    8838.0   48259.0  1684374.0       0.0  582170.0   
246        NaN    NaN       NaN       NaN        NaN       NaN       NaN   
247        NaN    NaN       NaN       NaN        NaN       NaN       NaN   
248        NaN    NaN       NaN       NaN        NaN       NaN       NaN   
249        NaN    NaN       NaN       NaN        NaN       NaN       NaN   

        REPRB   SOTCB      TEPRB      TETCB   WDEXB     WDPRB     WDTCB  \
0     22754.

In [60]:
# Test querying features

# all_msns = clean_df.MSN.unique()
# print(all_msns)
# X = pd.DataFrame(index=range(250), columns=all_msns)
# data_pt_idx= 0
# clean_df_row_idx = 0
# for year in all_years:
#     for state in all_states:
#         # data point
#         for feature_idx in range(19):
#             X.at[data_pt_idx, all_msns[feature_idx]] = clean_df.at[clean_df_row_idx, "Amount"]
#             clean_df_row_idx += 1
#         data_pt_idx += 1
# print(X)

['CLPRB' 'GETCB' 'HYTCB' 'NCPRB' 'NGMPB' 'NUETB' 'PAPRB' 'REPRB' 'SOTCB'
 'TEPRB' 'TETCB' 'WDEXB' 'WDPRB' 'WDTCB' 'WSTCB' 'WWPRB' 'WYTCB' 'BDPRP'
 'ENPRP']
         CLPRB      GETCB      HYTCB      NCPRB       NGMPB      NUETB  \
0      17747.0      186.0    14621.0    16298.0    380887.0        0.0   
1      91902.0    92144.0   182481.0   438728.0     55421.0   258955.0   
2    1031466.0   144716.0    35611.0   117840.0       124.0  1331452.0   
3        212.0   128954.0    48229.0   615867.0   1495858.0        0.0   
4     211839.0  2352883.0  7525491.0        0.0     98852.0    98852.0   
..         ...        ...        ...        ...         ...        ...   
245  11034.744        0.0        0.0        0.0    7702.296        0.0   
246   3801.672        0.0        0.0  39076.464  158715.816        0.0   
247        0.0   3466.512        0.0        0.0   48240.696        0.0   
248   5014.632  112549.92  80237.304  40940.592    2694.048        0.0   
249        0.0        0.0  112

In [73]:
# Create feature input matrix

X = pd.read_csv("flipped.csv")
# print(X)

sorted_X = X.sort_values(["Year", "State"], ascending=[True, True])
print(sorted_X)



    State  Year   BDPRP      CLPRB    ENPRP     GETCB     HYTCB     NCPRB  \
0      AK  2015     4.0    17747.0      0.0     186.0   14621.0   16298.0   
5      AL  2015   356.0   331420.0      0.0     141.0   91902.0   92144.0   
10     AR  2015   815.0     1819.0      0.0     808.0   33261.0   34194.0   
15     AZ  2015     2.0   146450.0   1154.0     345.0   60906.0  113693.0   
20     CA  2015   747.0        0.0   4650.0  112878.0  128681.0  567365.0   
..    ...   ...     ...        ...      ...       ...       ...       ...   
229    VT  2019     0.0        0.0      0.0      29.0   11906.0   18063.0   
234    WA  2019  1808.0        0.0      0.0    1136.0  587893.0  650831.0   
239    WI  2019   643.0        0.0  15000.0     615.0   23518.0   42338.0   
244    WV  2019     0.0  2422224.0      0.0      32.0   15192.0   29912.0   
249    WY  2019     0.0  4828513.0      0.0     663.0    8838.0   48259.0   

         NGMPB     NUETB  ...     REPRB     SOTCB      TEPRB      TETCB  \


In [25]:
print(alaska_df)

      Unnamed: 0    MSN StateCode  Year    Amount   State  \
0              4  CLPRB        AK  2015   17747.0  Alaska   
1             10  GETCB        AK  2015     186.0  Alaska   
2             11  HYTCB        AK  2015   14621.0  Alaska   
3             12  NCPRB        AK  2015   16298.0  Alaska   
4             13  NGMPB        AK  2015  380887.0  Alaska   
...          ...    ...       ...   ...       ...     ...   
4500           9  ENPRP        AK  2015       0.0  Alaska   
4550        1528  ENPRP        AK  2016       0.0  Alaska   
4600        3047  ENPRP        AK  2017       0.0  Alaska   
4650        4566  ENPRP        AK  2018       0.0  Alaska   
4700        6085  ENPRP        AK  2019       0.0  Alaska   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804                      16.0                3345612.0  
2               35.027804     

In [26]:
numRows = 5
numCols = 29

x_comb = pd.DataFrame(index=range(numRows), columns=range(numCols))
x_comb.columns = alaska_df["MSN"][0:numCols]

# x = pd.DataFrame(index=range(numRows), columns=range(numCols))
# x.columns = alaska_df["MSN"][0:numCols]
# x_test = pd.DataFrame(index=range(4, 5), columns=range(numCols))
# x_test.columns = alaska_df["MSN"][0:numCols]
# print(x.columns)
avgs = [[0] * numCols] * numRows
for row_idx in range(int(alaska_df.shape[0])):
    yr_idx = int(row_idx / numCols)
    x_comb.at[yr_idx, x_comb.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]
    # if row_idx < int(alaska_df.shape[0] * 4 / 5):
    #     x.at[yr_idx, x.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]
    # else:
    #     x_test.at[yr_idx, x.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]

norm_x_comb = (x_comb - x_comb.mean())/x_comb.std()

norm_x = norm_x_comb.head(4)
norm_x_test = norm_x_comb.tail(1)

# print(x)
# print(x_test)
print(norm_x)
print(norm_x_test)


MSN     BDFDB     BDPRP     BFFDB     BFPRP     CLPRB     CLPRK     CLPRP  \
0   -0.898188 -0.898173 -0.733353 -0.738738 -0.579055 -0.128243 -0.526746   
1   -0.296652 -0.296501 -0.586968 -0.586262   1.51242   0.94301  1.407429   
2    1.252345  1.251893 -0.210018 -0.193868 -0.732947  0.316033 -0.785824   
3    -0.90489 -0.905108 -0.210173 -0.220233  0.540098   0.51974   0.70079   

MSN     COPRK     EMFDB     ENPRP  ...     REPRB     SOTCB     TEPRB  \
0    0.350264 -0.580166 -0.579933  ... -0.677567 -0.542169  0.167092   
1    0.811137 -0.580166 -0.579933  ... -0.224796 -0.539738 -0.289706   
2    0.903312 -0.580166 -0.579933  ... -0.474328 -0.539675 -0.479445   
3   -0.663658  0.011308   0.01037  ... -0.388164 -0.140189 -1.017501   

MSN     TETCB     WDEXB     WDPRB     WDTCB     WSTCB     WWPRB     WYTCB  
0   -0.667942 -0.447214 -0.979919 -0.964505 -0.510625 -1.012773 -0.448595  
1   -0.224551 -0.447214  1.401078  1.419738 -0.454722  1.158346 -0.476136  
2   -0.517672  1.788854  

In [32]:
reg = linear_model.LinearRegression()
reg.fit(norm_x, y)

LinearRegression()

In [56]:
lasso_reg = Lasso(0.01)

In [57]:
lasso_reg.fit(norm_x, y)

Lasso(alpha=0.01)

In [58]:
lasso_train_score = lasso_reg.score(norm_x, y)
print(lasso_train_score)

0.9999999999999981


In [59]:
y_pred_lasso = lasso_reg.predict(norm_x_test)
print(y_pred_lasso)

[44282914.96467288]


In [60]:
print(lasso_reg.coef_)
print(lasso_reg.intercept_)

[ 1.50974904e+06 -1.35226473e+03  2.45032759e+07 -5.31672615e+05
 -3.91997302e+05 -2.72634955e+06  1.42367108e+06 -4.65758728e+06
 -1.01021503e+06 -0.00000000e+00 -1.26510818e+07 -4.04209822e+06
  2.07693012e+06 -1.02497433e+05  3.19311881e+05  5.32550054e+04
 -2.28053508e+05 -2.62631040e+04  1.22792058e+01 -4.48191328e+05
  1.54834116e+06  4.25570521e+04 -2.39660431e+05 -5.12497658e+03
 -2.07445485e+04 -9.44101376e+01  1.83503449e+04 -6.02462601e+02
  3.22705908e+05]
[16324509.79293481]


In [61]:
print(reg.intercept_)
print(reg.coef_)

[9304626.40874828]
[[  446070.88214614   445485.53466719   707433.75430459   706586.72209353
   -517087.19220269    51579.55147729  -310055.3419393  -1293046.64744555
    674952.8973394    673617.50995882    11148.79212505   149153.12733561
    258908.56090911  -453317.75058139 -1943887.78525199  -341171.8581966
    518574.50640231 -1552057.15946725 -1549762.78107775    46353.68936528
    457809.92894808 -1260433.68622962    59272.21460338   987989.40591331
  -1445936.0672543  -1484910.13242291    39043.89360253 -1300650.42063566
     81084.47730676]]


In [37]:
y_pred = reg.predict(norm_x_test)
print(y_pred)
print(y)

[[9183498.04374109]]
  TotalAmountofAssistance
0               3345612.0
1               2247734.0
2              13509662.0
3              18236626.0


In [41]:
mse = metrics.mean_squared_error([[13734021]], [[9183498.04374109]])
print(mse)

20707259175439.33
